In [1]:
import sys
import os
from algorithms.scheduling import FifoAlgorithm
from core.simulation import Simulation
import config.data as test_data 

In [2]:
os.chdir('..')
# Confirm we are in the topsim root directory - config.data has path maps relative to this 
os.getcwd()

'/home/rwb/github/topsim'

In [3]:
import simpy
import time
import os
workflow_file = None
event_file = 'sim.trace'
planning_algorithm = 'heft'
env = simpy.Environment()
tmax = 36  # for starters, we will define telescope configuration as simply number of arrays that exist
salgorithm = FifoAlgorithm()
vis = True

simulation = Simulation(env,
                       test_data.telescope_config,
                        tmax,
                        test_data.machine_config,
                        salgorithm,
                        'heft',
                        event_file,
                        vis)

events = []
not_started = True
start = False


INFO:bokeh.server.server:Starting Bokeh server version 1.3.4 (running on Tornado 6.0.3)


In [4]:
import time
import simpy

# Setup bokeh imports
from bokeh.application import Application
from bokeh.application.handlers.function import FunctionHandler
from bokeh.models import LinearColorMapper
from bokeh.models.widgets import Button, DataTable, TableColumn
from bokeh.server.server import Server
from bokeh.layouts import layout
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure

import logging
logger = logging.getLogger(__name__)


In [5]:
def make_document(doc):
    # source = ColumnDataSource({'x': [], 'y': [], 'color': []})
    # x = []
    # y = []

    colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
    mapper = LinearColorMapper(palette=colors, low=0, high=1)
    button = Button(label="Start", button_type="success")
    # create three plots

    plot = {
        'time': [env.now],
        'running': [len(simulation.cluster.running_tasks)],
        'finished': [len(simulation.cluster.finished_tasks)]
    }

    plotdata = ColumnDataSource(plot)

    def update_start():
        if not start:
            start = True

    p = figure(plot_width=400, plot_height=400)
    p.line(x='time', y='running', legend_label='Running tasks', alpha=0.2, line_width=3, color='navy', source=plotdata)
    p.line(x='time', y='finished', legend_label='Finished tasks', alpha=0.8, line_width=2, color='orange', source=plotdata)
    p.xaxis.axis_label = 'Time'
    p.yaxis.axis_label = 'Tasks'

    def update_plot():
        if env:
            updata = {
                'time': [env.now],
                'running': [len(simulation.cluster.running_tasks)],
                'finished': [len(simulation.cluster.finished_tasks)]
            }
            plotdata.stream(updata)

    """
    SETUP TABLE 
    """
    columns = [
        TableColumn(field="name", title="Name"),
        TableColumn(field="start", title="Start Time"),
        TableColumn(field="demand", title="No. Antennas"),
        TableColumn(field="duration", title="Observation Length"),
        TableColumn(field="running", title="Running Status")
    ]

    sourcedata = dict(name=[], start=[], duration=[], demand=[], running=[])
    tablesource = ColumnDataSource(data=sourcedata)
    for observation in simulation.telescope.observations:
        sourcedata['name'].append(observation.name)
        sourcedata['start'].append(observation.start)
        sourcedata['duration'].append(observation.duration)
        sourcedata['demand'].append(observation.demand)
        sourcedata['running'].append(observation.status)
    data_table = DataTable(source=tablesource, columns=columns, width=600, height=280)

    def update_table():
        if env:
            for i, observation in enumerate(simulation.telescope.observations):
                dictionary = {"running": [(i, observation.status)]}
                tablesource.patch(dictionary)

    doc.add_periodic_callback(update_plot, 500)
    doc.add_periodic_callback(update_table, 500)
    button.on_click(update_start)

    # fig.xgrid.grid_line_color = None
    # fig.x_range.start = 0
    # fig.x_range.end = 350
    # fig.legend.orientation = "horizontal"
    # fig.legend.location = "top_center"

    doc.title = "TopSim Dashboard"
    doc.add_root(layout([button], [p, data_table]))
    return doc

In [6]:
logger.debug('Opening Bokeh application on http://localhost:5070/')
apps = {'/': Application(FunctionHandler(make_document))}
server = Server(apps, port=5070)

INFO:bokeh.server.server:Starting Bokeh server version 1.3.4 (running on Tornado 6.0.3)
ERROR:tornado.application:Uncaught exception GET / (127.0.0.1)
HTTPServerRequest(protocol='http', host='localhost:5070', method='GET', uri='/', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "/home/rwb/github/topsim/venv/lib/python3.7/site-packages/tornado/web.py", line 1699, in _execute
    result = await result
  File "/home/rwb/github/topsim/venv/lib/python3.7/site-packages/tornado/gen.py", line 742, in run
    yielded = self.gen.throw(*exc_info)  # type: ignore
  File "/home/rwb/github/topsim/venv/lib/python3.7/site-packages/bokeh/server/views/doc_handler.py", line 55, in get
    session = yield self.get_session()
  File "/home/rwb/github/topsim/venv/lib/python3.7/site-packages/tornado/gen.py", line 735, in run
    value = future.result()
  File "/home/rwb/github/topsim/venv/lib/python3.7/site-packages/tornado/gen.py", line 742, in run
    yielded = self.gen

In [ ]:
server.start()

In [ ]:
simulation.start(runtime=-1)